In [37]:
from astropy import constants as const
from astropy import units as u
import math
from astropy.io import fits
from astropy.wcs import WCS
# from astropy.utils.data import get_pkg_data_filename
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks


from astropy.coordinates import SkyCoord  
from astropy.coordinates import FK5  
from photutils.aperture import SkyEllipticalAperture, SkyRectangularAperture
from photutils.aperture import aperture_photometry


## Importing fits files

In [ ]:
## [INPUT] filename of the H2CO fits cube
dir_data = '/Users/donglinwu/Desktop/College/Research/'
filename = dir_data+'/HOPS198_Data/HOPS198_H2CO_Tp12m7m_Final_Combine_pbcor.fits'

hdul = fits.open(filename)
hdul.info()

hdu = hdul[0]
cube_header = hdu.header
cube_data = hdu.data

## [EDIT] change it according to the axes of the fits file
wcs = WCS(cube_header)
wcs2d = wcs[0,:,:] 
wcsv = wcs[:,0,0]

In [39]:
## Extracts the velocity (spectral axis)
## [EDIT] change it according to the number of axes of the fits cube
crval3 = cube_header['CRVAL3']
crpix3 = cube_header['CRPIX3']
cdelt3 = cube_header['CDELT3']
## v_H2CO: velocity corresponding to each of the channels in meter per second
v_H2CO = (crval3 + ((np.arange(cube_data.shape[0]) + 1) - crpix3) * cdelt3)


In [40]:
## [FUNCTION] extracts noise part of the spectrum
def noise_from_spectrum(spectrum, default_noise):
    # spectrum: the input spectrum
    # default_noise: a list of two indices; spectrum between these two indices is the useful
    #                spectrum outside these two indices is pure noise, which will be extracted by this function
    #                e.g. default_noise = [20, 80], spectrum[:20] and spectrum[80:] will be extracted and combined 
    noise = []
    for i in range (len(spectrum)):
        if i<default_noise[0] or i>default_noise[1]:
            if math.isnan(spectrum[i]) == False:
                noise.append(spectrum[i])
    return noise

In [ ]:
## Masks the data to exclude the noises

## [EDIT]
## n_sigma: number of sigma the mask is applied
n_sigma = 5
## mol_index_noise: a list of two indices that define the range of the noise, see argument default_noise of the function noise_from_spectrum
mol_index_noise = [20, 130]

# data: spectral cube after the mask is applied
# noise: an image of the std noise level
data = np.zeros((cube_data.shape[0],cube_data.shape[1],cube_data.shape[2]),dtype=float)
noise = np.zeros((cube_data.shape[1],cube_data.shape[2]),dtype=float)



for xchan in range(cube_data.shape[1]):
    for ychan in range(cube_data.shape[2]):
        spectrum = cube_data[:,xchan,ychan]
        vec_noise = noise_from_spectrum(spectrum, [mol_index_noise[0],mol_index_noise[1]])
        noise[xchan,ychan] = np.std(vec_noise)
        Mask = np.greater(spectrum,n_sigma*np.std(vec_noise))
        data[:,xchan,ychan] = spectrum*Mask.astype(int)

In [42]:
## [INPUT] Imports the fits file for the mass of the outflow -> used as contours in plots
hdu_outflow = fits.open(dir_data+'/HOPS198/outflow/HOPS198_12CO_outflow_mass_order5_nrect_20.fits')[0]
M_outflow_pixel = hdu_outflow.data


In [ ]:
## [INPUT] Imports the fits file for the continuum -> used as contours in plots

from reproject import reproject_exact
hdu_cont = fits.open(dir_data+'HOPS198_Data/HOPS-198_Continuum_natural.thres0.11mJy.image.pbcor.fits')[0]
continuum, footprint2 = reproject_exact(hdu_cont, wcs[0,:,:])
# continuum = hdu_cont.data

In [44]:
## [INPUT] Imports the fits file for the disk -> used as contours in plots
hdu_disk = fits.open(dir_data+'/HOPS198_Data/VANDAM/HOPS-198_cont_robust2.pbcor.fits')[0]


## [EDIT] change it according to the axes of the fits file
disk_data_original = hdu_disk.data[0,0,:,:]


In [ ]:
## [OPTIONAL] 
## If the fits file of the disk has different resolution and WCS axes as the fits file of the other data, use this section
## This section finds the vertices of the contours of the disks under the WCS axes of the H2CO fits cube

## [EDIT] Change the levels of the corresponding contours 

plt.imshow(disk_data_original,origin='lower')
contour1 = plt.contour(disk_data_original, cmap='inferno',levels=[0.2*np.nanmax(disk_data_original)])
contour2 = plt.contour(disk_data_original, cmap='inferno',levels=[0.5*np.nanmax(disk_data_original)])
contour_data1 = contour1.collections[0].get_paths()
contour_data2 = contour2.collections[0].get_paths()
plt.xlim(400,600)
plt.ylim(400,600)

from astropy.wcs.utils import pixel_to_skycoord, skycoord_to_pixel
for path in contour_data1:
    # print(path.vertices)
    coords_contour1_disk_ALMA = skycoord_to_pixel(pixel_to_skycoord(path.vertices[:,0], path.vertices[:,1], WCS(hdu_disk.header)), WCS(hdu_cont.header)[:,:])
for path in contour_data2:
    # print(path.vertices)
    coords_contour2_disk_ALMA = skycoord_to_pixel(pixel_to_skycoord(path.vertices[:,0], path.vertices[:,1], WCS(hdu_disk.header)), WCS(hdu_cont.header)[:,:])



## Finding locations where there are two peaks

In [46]:
## [EDIT] 
## channel_min, channel_max: indices that define the range of channels in which the algorithm looks for the peaks
##                           it should take into consideration of the noises as well as contamination from the outflow
channel_min, channel_max = 58, 85

## height_peak: the minimum height required for the peak
##              should be adjusted according to the signal levels as well as signal-to-noise ratio
height_peak = 0.02


## bright_peak, dim_peak: a 2d array that records the velocity of the brighter or dimmer peak from the two peaks for spectrum of each of the pixels (in m/s)
bright_peak = np.zeros(shape=(cube_data.shape[2],cube_data.shape[1]))
dim_peak = np.zeros(shape=(cube_data.shape[2],cube_data.shape[1]))

## delta_vpeaks: a 2d array that records the velocity of the brighter peak subtracted from the velocity of the dimmer peak (in m/s)
##               if the blueshifted peak is brighter, this should be positive
deltav_peaks = np.zeros(shape=(cube_data.shape[2],cube_data.shape[1]))


for xchan in range(cube_data.shape[1]):
    for ychan in range(cube_data.shape[2]):
        peaks, heights = find_peaks(data[:, ychan, xchan], height=height_peak)
        heights = heights['peak_heights']
        peaks_new, heights_new = [], []
        for peak, height in zip(peaks, heights):
            if channel_min <= peak <= channel_max:
                peaks_new.append(peak)
                heights_new.append(height)
        peaks_new, heights_new = np.array(peaks_new), np.array(heights_new)
        if len(peaks_new) > 1:
            try:
                peak_first, height_first = peaks_new[np.argmax(heights_new)], heights[np.argmax(heights_new)]
                heights_f_removed = np.copy(heights_new)
                heights_f_removed[heights_f_removed == height_first] = -np.inf
                peak_second, height_second = peaks_new[np.argmax(heights_f_removed)], heights[np.argmax(heights_f_removed)]
                bright_peak[ychan, xchan] = v_H2CO[peak_first]
                dim_peak[ychan, xchan] = v_H2CO[peak_second]
                deltav_peaks[ychan, xchan] = v_H2CO[peak_second] - v_H2CO[peak_first]
            except:
                print(np.argmax(heights_new), heights[np.argmax(heights_new)])

deltav_peaks[deltav_peaks == 0] = np.nan
bright_peak[bright_peak == 0] = np.nan
dim_peak[dim_peak == 0] = np.nan


In [ ]:
## Plots the velocity of the brighter peak

semi_a = 150
bright_peak[bright_peak == 0] = np.nan
plt.imshow(bright_peak/1000, origin='lower', vmin=4.8, vmax=7)
plt.colorbar(label=r'$v_{\text{bright}}$ [km/s]')

plt.xlim(semi_a, 448-semi_a)
plt.ylim(semi_a, 448-semi_a)

In [ ]:
## Plots the velocity of the brighter peak subtracted from the velocity of the dimmer peak


import matplotlib.colors as mcolors
plt.figure()
plt.subplot(projection=wcs2d)

plt.imshow(deltav_peaks/1000, origin='lower',vmin=-0.6,vmax=0.6,cmap='coolwarm')
plt.colorbar(label=r'$\Delta v$ [km/s]')

## [EDIT] contours of the outflow, continuum and the disk data
plt.contour(np.greater(M_outflow_pixel, 0.3*np.nanstd(M_outflow_pixel)), cmap=mcolors.ListedColormap(['cyan']))
plt.contour(continuum, levels=[0.05*np.nanmax(continuum)],cmap='Reds')
plt.plot(coords_contour1_disk_ALMA[0],coords_contour1_disk_ALMA[1], color='black')
plt.plot(coords_contour2_disk_ALMA[0],coords_contour2_disk_ALMA[1], color='white')

plt.xlabel('RA')
plt.ylabel('DEC')

## [EDIT] the size of the plot generated
semi_a = 150
plt.xlim(semi_a, 448-semi_a)
plt.ylim(semi_a, 448-semi_a)

In [ ]:
## Plots the difference in velocity of the two peaks, for pixels where the blueshifted peak is brighter



## deltav_peaks_pos: difference in velocity of the two peaks, for pixels where the blueshifted peak is brighter
deltav_peaks_pos = np.copy(deltav_peaks)/1000
deltav_peaks_pos[deltav_peaks <= 0] = np.nan


import matplotlib.colors as mcolors
fig = plt.figure()
plt.subplot(projection=wcs2d)
im = plt.imshow(deltav_peaks_pos, origin='lower',vmin=0.3,vmax=0.6)
plt.colorbar(im, label=r'$\Delta v$ [km/s]')

## [EDIT] contours of the outflow, continuum and the disk data
plt.contour(np.greater(M_outflow_pixel, 0.3*np.nanstd(M_outflow_pixel)), cmap=mcolors.ListedColormap(['red']))
plt.contour(continuum, levels=[0.05*np.nanmax(continuum)],cmap='Reds')
plt.plot(coords_contour1_disk_ALMA[0],coords_contour1_disk_ALMA[1], color='black')
plt.plot(coords_contour2_disk_ALMA[0],coords_contour2_disk_ALMA[1], color='white')

plt.xlabel('RA')
plt.ylabel('DEC')

## [EDIT] the size of the plot generated
semi_a = 150
plt.xlim(semi_a, 448-semi_a)
plt.ylim(semi_a, 448-semi_a)


In [50]:
## Writes the difference in velocity of the two peaks, for pixels where the blueshifted peak is brighter, into a fits file



from astropy.io import fits

hdu_output = fits.PrimaryHDU()
hdu_output.data = deltav_peaks_pos

for i in range(len(list(hdu.header.keys()))):
    key = list(hdu.header.keys())[i]
    if '3' not in key and 'COMMENT' not in key and 'HISTORY' not in key:
        # print(key)
        hdu_output.header.update({key:hdu.header[key]})

hdu_output.header.update({'NAXIS':2})

# hdu_output.writeto('./HOPS198_H2CO_two_peaks_deltav_blue_brighter.fits')


In [ ]:
## Creates a mask, which only includes pixels where the spectrum shows two peaks and the blueshifted peak is brighter

mask_deltav_pos = np.copy(deltav_peaks_pos)
mask_deltav_pos[np.isnan(mask_deltav_pos) == False] = 1
plt.imshow(mask_deltav_pos, origin='lower')

## Fitting a two layer radiative transfer model

In [52]:
h = const.h.value
k = const.k_B.value
pi = math.pi

nu_o = cube_header['RESTFRQ']  #rest Frequency of H2CO(2-1) transition from header (in Hz)
bmaj = cube_header['BMAJ'] * 3600. #beam major axis (in arcsec)
bmin = cube_header['BMIN'] * 3600. #beam major axis (in arcsec)
dx = cube_header['CDELT1'] # length of pixel (in degrees)
dy = cube_header['CDELT2'] # width of pixel (in degrees)


In [ ]:
## [EDIT]
## level_continuum: the level that defines the "region of continuum", to which the radiative transfer model is applied, from the continuum data
##                  only pixels brighter than this level are included
## [REQUIRED]       the level is chosen so that the size of this region is approximately the same as the same of the beam
## note: there are various other ways to select a "region of continuum" 
level_continuum = 8.25e-3


## H2CO_cont: a 2d array which labels the pixels that are included in the "region of continuum"
## H2CO_continuum: the average spectrum of the "region of continuum"
##                 this will be the spectrum to which the radiative transfer model is applied


H2CO_continuum = np.zeros(hdu.data.shape[0])
H2CO_cont = np.zeros(shape=(hdu.data.shape[1], hdu.data.shape[2]))
n_pixels = 0
loc_cont = []
for i in range(-10, 10):
    for j in range(-10, 10):
        if np.max(continuum[224+i, 224+j]) >= level_continuum:
            n_pixels += 1
            # H2CO_continuum += data[:, 224+i, 224+j]
            H2CO_continuum += hdu.data[:, 224+i, 224+j]
            H2CO_cont[224+i, 224+j] = 1
            loc_cont.append((i,j))
print(n_pixels)
H2CO_continuum = H2CO_continuum/n_pixels


In [ ]:
## Plots the continuum data and the "region of continuum" 

plt.subplot(projection=wcs2d)
plt.imshow(continuum)
plt.contour(H2CO_cont, cmap='Accent')
# plt.contour(loc_two_peaks)
plt.xlim(200,448-200)
plt.ylim(200,448-200)

In [ ]:
## Computes the ratio of the area of the "region of continuum" selected to the area of the beam
## Ideally, it should be close to 1

A_continuum = abs(n_pixels*dx*3600*dy*3600)
A_beam = pi*bmaj*bmin/4
print(A_continuum/A_beam) 

In [56]:
## [FUNCTION] Converts the intensity Jy/beam to brightness temperautre (K)

def T_mb(I, nu, bmaj, bmin):
    # This function changes the intensity from Jy/beam to brightness temperautre (K)
    # I: intensity in Jy/beam
    # nu: frequency in Hz
    # bmin: beam minor axis (arcsec) 
    # bmax: beam major axis (arcsec)
    return 1.222*1e6*I/(((nu/1e9)**2)*bmaj*bmin) #K/beam

In [57]:
## T_H2CO_cont: intensity (in brightness temperature) of the average H2CO spectrum of the region selected
T_H2CO_cont = T_mb(H2CO_continuum, nu_o, bmaj, bmin)

In [58]:
## [OPTIONAL] 
## Computes the H2CO spectrum and std for each of the pixel in the region selected
## Not useful in most case

# vec_T_H2CO_cont = np.zeros(shape=(n_pixels,hdu.data.shape[0]))
# vec_T_H2CO_std = np.zeros(shape=hdu.data.shape[0])
# for (i,j), index in zip(loc_cont, range(n_pixels)):
#     vec_T_H2CO_cont[index] = T_mb(hdu.data[:, 224+i, 224+j], nu_o, bmaj, bmin)
# for index_v in range(hdu.data.shape[0]):
#     vec_T_H2CO_std[index_v] = np.std(np.transpose(vec_T_H2CO_cont)[index_v])
# plt.plot(v_H2CO, vec_T_H2CO_std)

In [ ]:
## [PLOTS] plots the average H2CO spectrum (in brightness temperature)
## Note: this is used to find the range of channels that the fit to the model should be applied to
##       the width of the spectrum should not be too large (if it is large, it is likely that there is contamination which should be excluded)
##       it is also used to identify the three important channels in this spectrum: the two peaks and the trough between
##       sometimes, these three points are assigned extra weight in the fit 

plt.plot(v_H2CO/1000, T_H2CO_cont)
# plt.plot(v_H2CO[:35]/1000, T_H2CO_cont[:35])
# plt.plot(v_H2CO[110:]/1000, T_H2CO_cont[110:])
plt.plot(v_H2CO[60:82]/1000, T_H2CO_cont[60:82], color='red')
plt.scatter(v_H2CO[66]/1000, T_H2CO_cont[66])
plt.scatter(v_H2CO[69]/1000, T_H2CO_cont[69])
plt.scatter(v_H2CO[75]/1000, T_H2CO_cont[75])
plt.xlabel(r'$v_{\text{LSR}}$ [km/s]')
plt.ylabel(r'$T_b$ [k]')

In [60]:
## [FUNCTION] This function defines the model
##            It outputs the expected brightness temperature according to the model
##            The model can be found at: DOI 10.1086/323854

## J_b: the background temperature: the CMB temperature
J_b = 2.725

## phi: fraction of the telescope beam area filled by the source (the source from the disk data)
phi = 0.5

def T_B_fit(v, v_sys, J_f, J_r, T_c, v_in, sigma, tau_0):
    # v: velocity of the channel evaluated
    # v_sys: system velocity, or velocity of the cloud; it can be a free parameter or a fixed parameter
    # J_f: temperature of the front layer
    # J_r: temperature of the rear layer
    # T_c: the blackbody temperature at the frequency of the line
    # v_in: infall velocity
    # sigma: velocity dispersion of the line
    # tau_0: optical thickness of the line
    
    T_o = h*nu_o/k
    J_c = T_o/(np.exp(T_o/T_c)-1)
    J_cr = phi*J_c + (1-phi)*J_r
    tau_f = tau_0*np.exp(-pow(v - v_in - v_sys,2)/(2*pow(sigma,2)))
    tau_r = tau_0*np.exp(-pow(v + v_in - v_sys,2)/(2*pow(sigma,2)))
    return (J_f - J_cr)*(1-np.exp(-tau_f))+(1-phi)*(J_r-J_b)*(1-np.exp(-tau_r-tau_f))

IMPORTANT REMARK: The following part is preliminary.  

In [61]:
## [EDIT]
## index_low, index_high: indices that define the range of channels that the fit to the model should be applied to
##                        it is important to exclude contamination from the outflow using these two indices
index_low,index_high = 60, 82


## [FUNCTION]
## Calculates the sum of difference between the expected surface temperature of the model with a given set of parameters
##            with the actual surface temperature measured by the data
##            

indices_extra_weight = [66, 69, 75]
extra_weight = 3

def func_fit(v_sys, J_f, J_r, T_c, v_in, sigma, tau_0):
    vdata, Tdata = v_H2CO, T_H2CO_cont
    mean_square = np.zeros(len(vdata))
    T_fit = T_B_fit(vdata, v_sys, J_f, J_r, T_c, v_in, sigma, tau_0)
    for i in range(index_low,index_high):
        mean_square[i] = pow(abs(T_fit[i]-Tdata[i]), 2) 
    if J_f < J_r:
        return sum(mean_square) + sum([mean_square[index_w] for index_w in indices_extra_weight])*extra_weight
    else:
        return sum(mean_square) + sum([mean_square[index_w] for index_w in indices_extra_weight])*extra_weight + 10


In [ ]:
import numpy as np
import emcee
import matplotlib.pyplot as plt

## Runs MCMC to fit the radiative transferl model to the observed surface temperature
## Uses uniform prior
## The system velocity can be a free parameter or a fixed parameter: if it is fixed, the model sometimes performs better

## [EDIT]
## Initial guess for the parameters: system velocity, J_f, J_r, T_c, infall velocity, velocity dispersion, optical thickness
initial_params = [5600, 29, 40, 30, 350, 260, 0.1]
bounds = np.array([
    [5595, 5605],   # v_sys
    [   10,  80],   # J_f
    [   10,  80],   # J_r
    [   10,  100],   # T_c
    [   0, 800],   # v_in
    [   0, 600],   # sigma
    [   0,    0.2],   # tau_0
])
n_dim = len(initial_params)

## Define a uniform prior 
def log_prior(params):
    low, high = bounds[:, 0], bounds[:, 1]
    if np.any(params < low) or np.any(params > high):
        return -np.inf
    return 0.0

## Define the cost function
def log_likelihood(params):
    cost = func_fit(*params)
    return -0.5 * cost

## Define the posterior distribution
def log_posterior(params):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params)

## Set up the sampler

## [EDIT]
## Number of walkers: the number of parallel MCMC chains exploring parameter space.
n_walkers = 64  

## Number of steps: how many iterations each walker will take (the length of each chain).
n_steps = 5000  

## Step size: controls the scale of the initial perturbation around your starting guess.
##            A value of 0.1 means each walker is placed in a Gaussian “ball” of radius ~10% of initial_params.
step_size = 0.1  


pos = initial_params + step_size * np.array(initial_params) * np.random.randn(n_walkers, n_dim)
sampler = emcee.EnsembleSampler(n_walkers, n_dim, log_posterior)
sampler.run_mcmc(pos, n_steps, progress=True)
samples = sampler.get_chain(flat=True)

## Plot the histograms of the distribution and store the means and highest peaks of the distributions of the parameters
param_names = ["v_sys","J_f","J_r","T_c","v_in","sigma","tau_0"]

## param_mean: means of the parameters
## param_peak: highest peak of the distributions of the parameters
param_mean, param_peak = [], []

for i, name in enumerate(param_names):
    param_samples = samples[:, i]
    mean, std = param_samples.mean(), param_samples.std()
    param_mean.append(mean)
    print(f"{name:6s} → Mean = {mean:.2f}, Std = {std:.2f}")

    # histogram + peak
    low, high = bounds[i]
    counts, edges = np.histogram(param_samples,
                                 bins=30,
                                 density=True,
                                 range=(low, high))
    centers = 0.5*(edges[:-1] + edges[1:])
    peak_loc = centers[np.argmax(counts)]
    param_peak.append(peak_loc)

    # plot
    plt.figure()
    plt.hist(param_samples,
             bins=30,
             density=True,
             range=(low, high),
             alpha=0.6)
    plt.title(f"{name}  (peak @ {peak_loc:.2f})")
    plt.xlabel(name)
    plt.ylabel("Density")
    plt.show()


In [ ]:
## Plots the model spectrum using peaks of the distributions of parameters and compares it with the measured spectrum 
## [IMPORTANT] Sometimes, the spectrum using the highest peak might deviate in terms of height.
##             This is often due to the optical depths being weakly constrained (sometimes two peaks are shown in the distribution).
##             The mean optical depth along with the highest peak of the other parameters often produce the best spectrum.


T_H2CO_fit = T_B_fit(v_H2CO,*param_peak) 
T_H2CO_adjusted = T_B_fit(v_H2CO,*param_peak[:len(param_peak)-1], param_mean[-1]) 
plt.plot(v_H2CO, T_H2CO_cont, label='Observed')
plt.plot(v_H2CO, T_H2CO_fit, label='Model (all peaks)')
plt.plot(v_H2CO, T_H2CO_adjusted, label='Model (mean for optical depth)')
plt.ylabel('T [K]')
plt.xlabel('v [m/s]')
plt.legend()